### 1️⃣ Load & Explore the Data

In [174]:
import pandas as pd

# Load your csv
df = pd.read_csv("cleaned_merged_seasons_team_aggregated.csv")

# Quick look at the data
print(df.head())
print(df.columns)
print(df.info())
print(df.describe())

C:\Users\user\AppData\Local\Temp\ipykernel_10472\590039501.py:4: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cleaned_merged_seasons_team_aggregated.csv")


  season_x                name position team_x  assists  bonus  bps  \
0  2016-17     Aaron Cresswell      DEF    NaN        0      0    0   
1  2016-17        Aaron Lennon      MID    NaN        0      0    6   
2  2016-17        Aaron Ramsey      MID    NaN        0      0    5   
3  2016-17  Abdoulaye Doucouré      MID    NaN        0      0    0   
4  2016-17   Abdul Rahman Baba      DEF    NaN        0      0    0   

   clean_sheets  creativity  element  ...  transfers_in  transfers_out  value  \
0             0         0.0      454  ...             0              0     55   
1             0         0.3      142  ...             0              0     60   
2             0         4.9       16  ...             0              0     80   
3             0         0.0      482  ...             0              0     50   
4             0         0.0       80  ...             0              0     55   

   was_home  yellow_cards GW  points  team_goals_scored team_goals_conceded  \
0     F

### 2️⃣ Data Cleaning
- Fill missing numeric values with 0 or median.

- Drop or fix rows with missing position or team_x.

- Ensure value is numeric and consistent (in millions).

- Convert categorical columns (position, team_x, opp_team_name) to one-hot encoding or numeric labels if needed.


### Fill / Drop strategy

- Numeric stats (points, team_goals_*): fill with 0 or median

- Categorical (team_x): fill missing teams by looking up the same player in other rows.

In [175]:
df.isna().sum()

season_x                   0
name                       0
position                   0
team_x                 20034
assists                    0
bonus                      0
bps                        0
clean_sheets               0
creativity                 0
element                    0
fixture                    0
goals_conceded             0
goals_scored               0
ict_index                  0
influence                  0
kickoff_time               0
minutes                    0
opponent_team              0
opp_team_name              0
own_goals                  0
penalties_missed           0
penalties_saved            0
red_cards                  0
round                      0
saves                      0
selected                   0
team_a_score               0
team_h_score               0
threat                     0
total_points               0
transfers_balance          0
transfers_in               0
transfers_out              0
value                      0
was_home      

In [176]:
# Fill numeric missing values with 0

df["points"] = df["points"].fillna(0)
df["team_goals_scored"] = df["team_goals_scored"].fillna(0)
df["team_goals_conceded"] = df["team_goals_conceded"].fillna(0)
df["team_goals_diff"] = df["team_goals_diff"].fillna(0)

# value is in tenths of millions
df["value"] = df["value"] / 10

# Create a dictionary mapping player name → most common team
# .mode() gives the most frequent value in that Series.
# If there’s a tie, it can return multiple values (a Series).
# x is a Series of all team_x values for one player. 
player_team_map = df.groupby("name")["team_x"].agg(lambda x : x.mode().iloc[0]
                                    if not x.mode().empty else None).to_dict()

# Fill missing team_x values using this mapping
df["team_x"] = df.apply(
    lambda row: player_team_map[row['name']] if pd.isna(row['team_x']) else row['team_x'],axis=1)


In [177]:
df.isna().sum()

season_x               0
name                   0
position               0
team_x                 0
assists                0
bonus                  0
bps                    0
clean_sheets           0
creativity             0
element                0
fixture                0
goals_conceded         0
goals_scored           0
ict_index              0
influence              0
kickoff_time           0
minutes                0
opponent_team          0
opp_team_name          0
own_goals              0
penalties_missed       0
penalties_saved        0
red_cards              0
round                  0
saves                  0
selected               0
team_a_score           0
team_h_score           0
threat                 0
total_points           0
transfers_balance      0
transfers_in           0
transfers_out          0
value                  0
was_home               0
yellow_cards           0
GW                     0
points                 0
team_goals_scored      0
team_goals_conceded    0


### 3️⃣ Encode Categorical Columns
position, team_x, opp_team_name → one-hot or label encoding

In [178]:
df['position']=df['position'].replace('GKP','GK')

In [179]:
# One-hot encode positions
df = pd.get_dummies(df , columns=['position'], prefix='pos')

# Optionally one-hot encode team names
df = pd.get_dummies(df,columns=['team_x','opp_team_name'],prefix=['team','opp'])

In [180]:
df['pos_GK'].sum()

14095

In [182]:
df.head()

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,opp_Sheffield Utd,opp_Southampton,opp_Spurs,opp_Stoke,opp_Sunderland,opp_Swansea,opp_Watford,opp_West Brom,opp_West Ham,opp_Wolves
0,2016-17,Aaron Cresswell,0,0,0,0,0.0,454,10,0,...,False,False,False,False,False,False,False,False,False,False
1,2016-17,Aaron Lennon,0,0,6,0,0.3,142,3,0,...,False,False,True,False,False,False,False,False,False,False
2,2016-17,Aaron Ramsey,0,0,5,0,4.9,16,8,3,...,False,False,False,False,False,False,False,False,False,False
3,2016-17,Abdoulaye Doucouré,0,0,0,0,0.0,482,7,0,...,False,True,False,False,False,False,False,False,False,False
4,2016-17,Abdul Rahman Baba,0,0,0,0,0.0,80,10,0,...,False,False,False,False,False,False,False,False,True,False


### 4️⃣ Convert datetime
kickoff_time can be converted if you plan to include recency/fixture timing

In [183]:
df["kickoff_time"].head()

0    2016-08-15T19:00:00Z
1    2016-08-13T14:00:00Z
2    2016-08-14T15:00:00Z
3    2016-08-13T14:00:00Z
4    2016-08-15T19:00:00Z
Name: kickoff_time, dtype: object

In [184]:
df["kickoff_time"] = pd.to_datetime(df["kickoff_time"])

In [185]:
df['kickoff_time'].head()

0   2016-08-15 19:00:00+00:00
1   2016-08-13 14:00:00+00:00
2   2016-08-14 15:00:00+00:00
3   2016-08-13 14:00:00+00:00
4   2016-08-15 19:00:00+00:00
Name: kickoff_time, dtype: datetime64[ns, UTC]

### 5️⃣ Quick Sanity Check

In [186]:
print(df.info())
print("--------------------------------------------")
print(df.describe())
print("--------------------------------------------")
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126076 entries, 0 to 126075
Data columns (total 100 columns):
 #   Column               Non-Null Count   Dtype              
---  ------               --------------   -----              
 0   season_x             126076 non-null  object             
 1   name                 126076 non-null  object             
 2   assists              126076 non-null  int64              
 3   bonus                126076 non-null  int64              
 4   bps                  126076 non-null  int64              
 5   clean_sheets         126076 non-null  int64              
 6   creativity           126076 non-null  float64            
 7   element              126076 non-null  int64              
 8   fixture              126076 non-null  int64              
 9   goals_conceded       126076 non-null  int64              
 10  goals_scored         126076 non-null  int64              
 11  ict_index            126076 non-null  float64            
 12  i

In [187]:
df.columns.values

array(['season_x', 'name', 'assists', 'bonus', 'bps', 'clean_sheets',
       'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time',
       'minutes', 'opponent_team', 'own_goals', 'penalties_missed',
       'penalties_saved', 'red_cards', 'round', 'saves', 'selected',
       'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW', 'points', 'team_goals_scored',
       'team_goals_conceded', 'team_goals_diff', 'pos_DEF', 'pos_FWD',
       'pos_GK', 'pos_MID', 'team_Arsenal', 'team_Aston Villa',
       'team_Bournemouth', 'team_Brentford', 'team_Brighton',
       'team_Burnley', 'team_Chelsea', 'team_Crystal Palace',
       'team_Everton', 'team_Fulham', 'team_Leeds', 'team_Leicester',
       'team_Liverpool', 'team_Luton', 'team_Man City', 'team_Man Utd',
       'team_Newcastle', 'team_Norwich', "team_Nott'm Fores

### let’s build the optimization-based FPL team picker (11 starters + bench)

In [188]:
import pandas as pd
import numpy as np 
from pulp import LpProblem,LpVariable,LpMaximize,lpSum,LpBinary,PULP_CBC_CMD


In [19]:
pip install pulp

  Using cached pulp-3.2.2-py3-none-any.whl.metadata (6.9 kB)
Using cached pulp-3.2.2-py3-none-any.whl (16.4 MB)
Note: you may need to restart the kernel to use updated packages.


### # --- 1) Target Params you can change

In [189]:
TARGET_SEASON = "2023-24"       # season to pick a team for
TARGET_GW = 10                  # gameweek to pick a team for
BUDGET_MILLIONS = 100.0         # FPL budget
VALUE_SCALE = 10.0              # your 'value' looks like 50 -> 5.0M, so divide by 10
BENCH_WEIGHT = 0.1              # tiny weight for bench expected points (optional)

### # Formation rules for starters (FPL valid formations)

In [22]:
MIN_DEF_STARTERS, MAX_DEF_STARTERS = 3, 5
MIN_MID_STARTERS, MAX_MID_STARTERS = 2, 5
MIN_FWD_STARTERS, MAX_FWD_STARTERS = 1, 3

###  --- 2) Expect the dataframe df to be loaded already

In [190]:
df.head()

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,opp_Sheffield Utd,opp_Southampton,opp_Spurs,opp_Stoke,opp_Sunderland,opp_Swansea,opp_Watford,opp_West Brom,opp_West Ham,opp_Wolves
0,2016-17,Aaron Cresswell,0,0,0,0,0.0,454,10,0,...,False,False,False,False,False,False,False,False,False,False
1,2016-17,Aaron Lennon,0,0,6,0,0.3,142,3,0,...,False,False,True,False,False,False,False,False,False,False
2,2016-17,Aaron Ramsey,0,0,5,0,4.9,16,8,3,...,False,False,False,False,False,False,False,False,False,False
3,2016-17,Abdoulaye Doucouré,0,0,0,0,0.0,482,7,0,...,False,True,False,False,False,False,False,False,False,False
4,2016-17,Abdul Rahman Baba,0,0,0,0,0.0,80,10,0,...,False,False,False,False,False,False,False,False,True,False


In [191]:
df.value.head()

0    5.5
1    6.0
2    8.0
3    5.0
4    5.5
Name: value, dtype: float64

### --- 3) Basic cleaning / helpers -------------------------------------------------


In [192]:
df.columns.values

array(['season_x', 'name', 'assists', 'bonus', 'bps', 'clean_sheets',
       'creativity', 'element', 'fixture', 'goals_conceded',
       'goals_scored', 'ict_index', 'influence', 'kickoff_time',
       'minutes', 'opponent_team', 'own_goals', 'penalties_missed',
       'penalties_saved', 'red_cards', 'round', 'saves', 'selected',
       'team_a_score', 'team_h_score', 'threat', 'total_points',
       'transfers_balance', 'transfers_in', 'transfers_out', 'value',
       'was_home', 'yellow_cards', 'GW', 'points', 'team_goals_scored',
       'team_goals_conceded', 'team_goals_diff', 'pos_DEF', 'pos_FWD',
       'pos_GK', 'pos_MID', 'team_Arsenal', 'team_Aston Villa',
       'team_Bournemouth', 'team_Brentford', 'team_Brighton',
       'team_Burnley', 'team_Chelsea', 'team_Crystal Palace',
       'team_Everton', 'team_Fulham', 'team_Leeds', 'team_Leicester',
       'team_Liverpool', 'team_Luton', 'team_Man City', 'team_Man Utd',
       'team_Newcastle', 'team_Norwich', "team_Nott'm Fores

In [193]:
df["pos_GK"].head()

0    False
1    False
2    False
3    False
4    False
Name: pos_GK, dtype: bool

In [194]:
df["pos_GK"].sum()

14095

GK DEF MID FWD FLAGS

In [195]:
pos_cols = set(df.columns)

is_gk = np.zeros(len(df), dtype=int)   
# Create a numpy array of zeros with the same length as the number of rows in df.
# Each entry will be 0 or 1 to indicate whether that player is a goalkeeper.

if 'pos_GK' in pos_cols:
    is_gk |= df['pos_GK'].astype(int).values  
# Take that column, convert it to integers (0 or 1), and OR (|=) it into is_gk.
# Example: if pos_GK = [1,0,0,0,...], then is_gk will become [1,0,0,0,...].


df['is_GK'] = is_gk


df['is_DEF'] = df['pos_DEF'].astype(int) if 'pos_DEF' in pos_cols else 0
df['is_MID'] = df['pos_MID'].astype(int) if 'pos_MID' in pos_cols else 0
df['is_FWD'] = df['pos_FWD'].astype(int) if 'pos_FWD' in pos_cols else 0

In [196]:
df['is_GK'].sum()

14095

- .astype(int) converts it into integers (still a Series).

- So :

is_gk |= df['pos_GKP'].astype(int).values


- is equivalent to

is_gk = is_gk | df['pos_GKP'].astype(int).values


- Since both sides are NumPy arrays of integers, this works element-wise:

0 | 0 = 0

0 | 1 = 1

1 | 0 = 1

1 | 1 = 1

In [197]:
df[['is_GK','is_DEF','is_MID','is_FWD']].head()

,is_GK,is_DEF,is_MID,is_FWD
0,0,1,0,0
1,0,0,1,0
2,0,0,1,0
3,0,0,1,0
4,0,1,0,0


### Sanity: exactly one position per row


In [198]:
df.shape

(126076, 104)

In [199]:
pos_sum = df[['is_GK','is_DEF','is_MID','is_FWD']].sum(axis=1)

# remove rows where position isn't unique
df = df[pos_sum == 1].copy()

In [200]:
df.shape

(126076, 104)

- .copy() → creates a fresh DataFrame copy of the filtered result.

- This avoids the SettingWithCopyWarning in Pandas when modifying later.

- It’s a safe practice: we now have a clean, independent DataFrame containing only valid players.

### --- 4) Build a simple "expected_points" for the target GW ----------------------
- We’ll use: last-3-GW average per player in the same season (form),
- with fallback to season average, then to overall career average.


In [201]:
# Ensure proper sorting
df.sort_values(['name','season_x','GW'],inplace=True)

In [202]:
df.head()

,season_x,name,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,...,opp_Sunderland,opp_Swansea,opp_Watford,opp_West Brom,opp_West Ham,opp_Wolves,is_GK,is_DEF,is_MID,is_FWD
20034,2020-21,Aaron Connolly,0,0,-3,0,0.3,78,7,2,...,False,False,False,False,False,False,0,0,0,1
20459,2020-21,Aaron Connolly,0,2,27,1,11.3,78,16,0,...,False,False,False,False,False,False,0,0,0,1
20998,2020-21,Aaron Connolly,0,0,2,0,12.1,78,19,2,...,False,False,False,False,False,False,0,0,0,1
21552,2020-21,Aaron Connolly,0,0,7,0,0.3,78,32,3,...,False,False,False,False,False,False,0,0,0,1
22114,2020-21,Aaron Connolly,1,0,13,0,10.3,78,40,0,...,False,False,False,False,False,False,0,0,0,1


- Voici ce qui se passe avec shift(1) en pandas :

    Ça décale toutes les valeurs d’une série ou d’une colonne vers le bas.

    La première ligne devient NaN, car il n’y a rien "au-dessus" d’elle.

    Les dernières lignes ne sont pas supprimées → elles restent, mais leur ancienne valeur a été décalée plus bas. Donc la dernière ligne va contenir NaN uniquement si tu décalais vers le haut (shift(-1)), pas avec shift(1).

    La derniere ligne n’a pas disparu, il a juste été décalé hors de la "fenêtre" visible quand tu as fait shift(1).

- En gros, pour calculer la moyenne d’un joueur à la journée j, on ne prend pas sa valeur actuelle, mais ses performances précédentes (on regarde en arrière).

In [203]:
# Season-wise rolling form (exclude current GW)
def last_n_avg(s, n=3):
    return s.shift(1).rolling(n, min_periods=1).mean()

- To avoid this error :

    (TypeError: incompatible index of inserted column with frame index) is very common when using groupby().apply(...) and then trying to assign the result back to the dataframe.

    groupby().apply(...) can return a Series with a MultiIndex ((name, season_x, row_index)) , while your dataframe df has just a normal RangeIndex or single index.

- Instead of .apply(...), use .transform(...):

    transform keeps the same index as the original dataframe.

    Perfect for adding new columns that align row by row.

In [204]:
# Total points of last 3 / 5 games

df['form_l3'] = df.groupby(['name','season_x'])['total_points'].transform(lambda s: last_n_avg(s,3))

df['form_l5'] = df.groupby(['name','season_x'])['total_points'].transform(lambda s: last_n_avg(s,5))

    🔹 .expanding(min_periods=1)

Imagine une "fenêtre" qui grandit au fur et à mesure.

Contrairement à .rolling(n), qui prend toujours les n dernières valeurs,
.expanding() prend toutes les valeurs depuis le début jusqu’à la ligne actuelle.

Donc season_avg_so_far = la moyenne cumulative des points d’un joueur dans la saison, sans compter le match actuel.

In [205]:
# Season average up to previous GW
df['season_avg_so_far'] = df.groupby(['name','season_x'])['total_points'].transform(
    lambda s: s.shift(1).expanding(min_periods=1).mean()
)

In [206]:
# Global (career) average up to previous row
df['career_avg_so_far']=df.groupby(['name'])['total_points']\
    .transform(lambda s: s.shift(1).expanding(min_periods=1).mean())

- Team signal (scaled goal difference this season to date — simple proxy of strength)
-  Use cumulative up to previous GW in the season if available, else 0

In [207]:
for col in ['team_goals_diff']:
    if col not in df.columns:
        df[col] = 0.0



- This is usually team goals scored − team goals conceded in a match (the goal difference).
- Example: if Manchester City wins 3–1, then team_goals_diff = +2. If they lose 0–2, then team_goals_diff = -2.
- For player X in season Y, at match i, this feature says:
“What was my team’s average goal difference in all games before this one?”

In [208]:
df['team_diff_so_far'] = df.groupby(['name','season_x'])['team_goals_diff']\
    .transform(lambda s: s.shift(1).expanding(min_periods=1).mean())

In [209]:
df['team_diff_so_far'].head(10)

20034         NaN
20459   -2.000000
20998   -0.500000
21552   -0.333333
22114   -0.750000
22701   -1.000000
23291   -1.166667
23888   -1.428571
24488   -1.625000
25089   -1.666667
Name: team_diff_so_far, dtype: float64

In [210]:
df['team_diff_so_far'].isna().sum()

3658

- Step by step:

td = df['team_diff_so_far'].fillna(0)

- The feature team_diff_so_far was calculated earlier (average goal difference of the player’s team up to now).

- Sometimes it has NaN (especially at the very beginning of a season when there’s no past data).

- fillna(0) replaces missing values with 0, meaning "neutral / no signal yet."

(td - td.mean())

- This centers the values around 0 by subtracting the mean of the column.

- If a team usually has a goal difference of +5, then a current +10 is "above average", while +2 is "below average".

/ (td.std() + 1e-6)

- Divides by the standard deviation to scale the values.

- This makes the feature comparable across different ranges (normalization).

+ 1e-6 is just a tiny number added to avoid division by zero errors.

Result → team_signal

- Now, the feature is standardized:

- Positive values (>0) mean the team is performing above average compared to itself.

- Negative values (<0) mean the team is performing below average.

- Because it’s normalized, most values fall between -1 and 1 (not always exactly, but roughly).

👉 Why do this?
Because ML models work better when features are normalized. Instead of giving the model raw numbers like team_diff_so_far = 15 or -8, we give it a relative signal that shows how strong/weak the team looks compared to average.

In [211]:
# Normalize team_diff_so_far roughly to ~[-1, 1]
# Because ML models work better when features are normalized.

td = df['team_diff_so_far'].fillna(0)
df['team_signal'] = (td-td.mean()) / (td.std() + 1e-6)


In [212]:
df['team_signal'].head(10)

20034    0.026745
20459   -0.193160
20998   -0.028231
21552   -0.009906
22114   -0.055719
22701   -0.083207
23291   -0.101533
23888   -0.130330
24488   -0.151928
25089   -0.156509
Name: team_signal, dtype: float64

- Expected points blend (tweak weights as you like)
- Priority: recent form > season avg > career avg, with a tiny team context signal

In [213]:
def blend(row) : 
    candidates = [
        row['form_l3'],
        row['season_avg_so_far'],
        row['career_avg_so_far']
    ]
    # Fill fallbacks
    vals = [v for v in candidates if pd.notnull(v)]
    base = np.average(vals,weights=[0.6,0.3,0.1][:len(vals)]) if len(vals) else 0.0
    return base + 0.15 * row['team_signal']

df['expected_points'] = df.apply(blend,axis=1).clip(lower=0.0)

- return base + 0.15 * row['team_signal'] → :

base is the weighted personal form of the player.

team_signal = some metric about the team’s strength (like average goal difference, or expected goals).

Adding 0.15 * team_signal means we also adjust the prediction by the team’s current trend → a weak player in a strong attacking team will still have more chances to score points.

- .clip(lower=0.0) → ensures expected points can’t go below 0 : 

Example: if base was very negative or team_signal negative (maybe team is in terrible form), we don’t want negative points prediction (nonsense).

So, everything < 0 becomes 0.0.

- ✅ Why this formula?

Most weight on short-term form.

Some stabilizer from season and career averages.

Small positive or negative adjustment from team strength.

Never predict below 0 points.

### --- 5) Candidate pool for the target GW ---------------------------------------


In [214]:
pool = df[(df['season_x'] == TARGET_SEASON) & (df['GW'] == TARGET_GW)].copy()

In [216]:
print(pool[['is_GK','is_DEF','is_MID','is_FWD']].sum())


is_GK      85
is_DEF    238
is_MID    314
is_FWD     94
dtype: int64


In [217]:
# If expected_points ended up NaN (e.g., brand-new player), fall back to small prior
pool['expected_points'] = pool['expected_points'].fillna(0.5)

In [218]:
pool['value'].head()

102939    4.5
103103    4.2
103196    4.5
102907    4.8
103368    5.0
Name: value, dtype: float64

In [219]:
pool['value'].isna().sum()

0

In [220]:
# Budget sanity
if pool['value'].isnull().any():
    pool['value'] = pool['value'].fillna(pool['value'].median())

### --- 6) Identify club columns for "max 3 per club" constraint -------------------


In [221]:
team_cols = [c for c in pool.columns if c.startswith('team_')]
if not team_cols:
    raise ValueError("No team_* one-hot columns found; needed for max-3-per-club constraint.")

# --- 7) Build the optimization model -------------------------------------------


- LpProblem = the optimization model container (the “problem” you give to the solver).

- "FPL_Team_Selection" = just a name for the model (useful in logs).

- LpMaximize = tells the solver we want to maximize some objective (e.g., total expected points). If you wanted a minimization problem you’d use LpMinimize.

- a model (also called a mathematical optimization problem).

#### Build the problem :
Think of prob (the LpProblem) as a container where you put:

1 objective function (what you want to optimize)

many constraints (rules the solution must follow)

In [265]:
# Problem : Maximization 
prob = LpProblem("FPL_Team_Selection",LpMaximize)


- You’ll create one or more decision variables per player.

- pool is the DataFrame of candidate rows (players for a given GW).

- idx is the list of row indices you’ll loop over to define variables and constraints.

In [223]:
pool.index

Index([102939, 103103, 103196, 102907, 103368, 103344, 102997, 102809, 103293,
       103044,
       ...
       103289, 103299, 102893, 102935, 103144, 102995, 103300, 103082, 103346,
       102923],
      dtype='int64', length=731)

In [224]:
idx = list(pool.index)

- LpVariable = defines a decision variable in PuLP (the unknown the solver decides).

- Creates a binary decision variable in_squad_i for each player i.

- Binary ⇒ takes value 0 or 1.

- Interpretation: in_squad_i = 1 if player i is selected in the 15-man squad; 0 otherwise.

- lowBound=0, upBound=1 plus cat=LpBinary both enforce binary. (Technically cat=LpBinary is enough; the bounds are redundant but harmless.)

- The comprehension {i: ... for i in idx} :

{
   idx[0]: LpVariable("in_squad_idx0", 0, 1, LpBinary),
   idx[1]: LpVariable("in_squad_idx1", 0, 1, LpBinary),
   ...
}

In [225]:
# Binary: in 15-man squad?
in_squad = {
    i: LpVariable(f"in_squad_{i}",lowBound=0,upBound=1,cat=LpBinary) for i in idx
}

- Another binary decision variable is_start_i for each player.

- Interpretation: is_start_i = 1 if player i is in the starting XI; 0 otherwise.

In [226]:
# Binary: in 11 starters?

is_start = {
    i: LpVariable(f"is_start{i}",lowBound=0,upBound=1,cat=LpBinary) for i in idx
}


-> `Objective`: maximize starters' expected points + small bench value

#### Add objective function (what you want to optimize) :
- += is just the syntax PuLP uses to add something into the container(prob).
The first time you do:

    prob += some_expression

- → PuLP assumes this is the objective function.


- is_start[i] and in_squad[i] : They aren’t 0 or 1 yet. At this stage they are just decision variables (LpVariable objects).

- The solver’s job is to later decide their values (0 or 1) such that the objective is maximized and all constraints are respected.

- For normal numbers → Python’s sum() or + is fine.

- For LpVariables → PuLP needs to construct a linear expression object, not a plain number.

- lpSum = ∑ (sigma in math).

- Normal operators + - * / are allowed, but instead of real math they build a symbolic expression the solver can understand.

- Even though is_start[i] is an LpVariable, PuLP overloads Python’s operators (+, -, *, /).

In [266]:
prob += lpSum(is_start[i]*pool.loc[i,'expected_points'] for i in idx) \
    + BENCH_WEIGHT*lpSum((in_squad[i]-is_start[i])*pool.loc[i,'expected_points'] for i in idx)

### Add Constraints

In [267]:
# 7.1 Link starters must be in squad

# if in_squad 0 is_start is 0 
# if in_squad 1 is_start is 0 or 1
for i in idx:
    prob += is_start[i] <= in_squad[i]


In [268]:
# 7.2 Squad size = 15; starters = 11

prob += lpSum(in_squad[i] for i in idx) == 15
prob += lpSum(is_start[i] for i in idx) == 11


In [269]:
# 7.3 Squad position counts: GK=2, DEF=5, MID=5, FWD=3

prob += lpSum(in_squad[i] * pool.loc[i,"is_GK"] for i in idx) == 2
prob += lpSum(in_squad[i] * pool.loc[i,"is_DEF"] for i in idx) == 5
prob += lpSum(in_squad[i] * pool.loc[i,"is_MID"] for i in idx) == 5
prob += lpSum(in_squad[i] * pool.loc[i,"is_FWD"] for i in idx) == 3

In [273]:
# 7.4 Starters formation constraints: 1 GK; DEF 3–5; MID 2–5; FWD 1–3

prob += lpSum(is_start[i] * pool.loc[i,'is_GK'] for i in idx) == 1
prob += lpSum(is_start[i] * pool.loc[i,'is_DEF'] for i in idx) >= MIN_DEF_STARTERS
prob += lpSum(is_start[i] * pool.loc[i,'is_DEF'] for i in idx) <= MAX_DEF_STARTERS
prob += lpSum(is_start[i] * pool.loc[i,'is_MID'] for i in idx) >= MIN_MID_STARTERS
prob += lpSum(is_start[i] * pool.loc[i,'is_MID'] for i in idx) <= MAX_MID_STARTERS
prob += lpSum(is_start[i] * pool.loc[i, 'is_FWD'] for i in idx) >= MIN_FWD_STARTERS
prob += lpSum(is_start[i] * pool.loc[i, 'is_FWD'] for i in idx) <= MAX_FWD_STARTERS


In [277]:
# 7.5 Budget

prob += lpSum(in_squad[i] * pool.loc[i,'value'] for i in idx) <= BUDGET_MILLIONS


In [233]:
pool[team_cols].isna().sum()

team_a_score           0
team_h_score           0
team_goals_scored      0
team_goals_conceded    0
team_goals_diff        0
team_Arsenal           0
team_Aston Villa       0
team_Bournemouth       0
team_Brentford         0
team_Brighton          0
team_Burnley           0
team_Chelsea           0
team_Crystal Palace    0
team_Everton           0
team_Fulham            0
team_Leeds             0
team_Leicester         0
team_Liverpool         0
team_Luton             0
team_Man City          0
team_Man Utd           0
team_Newcastle         0
team_Norwich           0
team_Nott'm Forest     0
team_Sheffield Utd     0
team_Southampton       0
team_Spurs             0
team_Watford           0
team_West Brom         0
team_West Ham          0
team_Wolves            0
team_diff_so_far       1
team_signal            0
dtype: int64

In [234]:
team_cols = [tc for tc in team_cols if tc not in ['team_diff_so_far','team_signal']]

In [235]:
pool[team_cols].isna().sum()

team_a_score           0
team_h_score           0
team_goals_scored      0
team_goals_conceded    0
team_goals_diff        0
team_Arsenal           0
team_Aston Villa       0
team_Bournemouth       0
team_Brentford         0
team_Brighton          0
team_Burnley           0
team_Chelsea           0
team_Crystal Palace    0
team_Everton           0
team_Fulham            0
team_Leeds             0
team_Leicester         0
team_Liverpool         0
team_Luton             0
team_Man City          0
team_Man Utd           0
team_Newcastle         0
team_Norwich           0
team_Nott'm Forest     0
team_Sheffield Utd     0
team_Southampton       0
team_Spurs             0
team_Watford           0
team_West Brom         0
team_West Ham          0
team_Wolves            0
dtype: int64

In [253]:
# 7.6 Max 3 per club (apply to the 15-man squad)
#for tcol in team_cols:
#    prob += lpSum(in_squad[i] * pool.loc[i, tcol] for i in idx) <=3

### --- 8) Solve -------------------------------------------------------------------
1) prob.solve(...)

- prob is your model (created with LpProblem).

- .solve() → tells PuLP: “Now hand this optimization model to a solver, let it compute the best solution, and store the result inside prob and its variables.”

- So after this line, every decision variable (in_squad[i], is_start[i], etc.) now has a chosen value (0 or 1).


2) PULP_CBC_CMD(...)

- This is a solver interface provided by PuLP.

- CBC = “Coin-or branch and cut” → an open-source optimization solver.

- PULP_CBC_CMD is just the Python wrapper class that lets you run CBC from PuLP.

👉 In short:
PULP_CBC_CMD = solver type (the actual algorithm that finds the solution).

3) msg=False

- Solvers normally print a lot of logs to the console (iterations, branching, etc.).

- msg=False → disables those messages → cleaner output.

- If you set msg=True, you’d see the solver’s progress printed.

In [278]:
_ = prob.solve(PULP_CBC_CMD(msg=False))

In [279]:
from pulp import LpStatus
print("Solver status:", LpStatus[prob.status])


Solver status: Optimal


### --- 9) Extract solution --------------------------------------------------------


In [280]:
pool['in_squad'] = [int(in_squad[i].value()) for i in idx]
pool['is_start'] = [int(is_start[i].value()) for i in idx]

squad = pool[pool['in_squad'] == 1].copy()
starters = squad[squad['is_start'] == 1].copy()
bench = squad[squad['is_start'] == 0].copy()

def pretty_columns(df_):
    cols = ['name','season_x','GW','value_m','expected_points','total_points','minutes']
    # add simple human position label
    df_ = df_.copy()
    df_['pos'] = np.select(
        [df_['is_GK']==1, df_['is_DEF']==1, df_['is_MID']==1, df_['is_FWD']==1],
        ['GK','DEF','MID','FWD']
    )
    keep = [c for c in cols if c in df_.columns]
    return df_[['pos'] + keep].sort_values(['pos','expected_points'], ascending=[True,False])

print("=== Starters (11) ===")
display(pretty_columns(starters))

print("\n=== Bench (4) ===")
display(pretty_columns(bench))

spent = squad['value'].sum()
exp_pts_start = starters['expected_points'].sum()
exp_pts_bench = bench['expected_points'].sum()
print(f"\nBudget spent: {spent:.1f} / {BUDGET_MILLIONS:.1f}M")
print(f"Expected points (starters): {exp_pts_start:.2f}")
print(f"Expected points (bench * {BENCH_WEIGHT}): {(BENCH_WEIGHT*exp_pts_bench):.2f} (raw bench {exp_pts_bench:.2f})")

=== Starters (11) ===


,pos,name,season_x,GW,expected_points,total_points,minutes
102981,DEF,Kieran Trippier,2023-24,10,7.989498,0,90
102754,DEF,Benjamin White,2023-24,10,6.619633,6,65
102924,DEF,Micky van de Ven,2023-24,10,6.491742,2,90
102953,DEF,Joachim Andersen,2023-24,10,6.189119,3,90
102789,FWD,Ollie Watkins,2023-24,10,10.623603,2,90
103118,FWD,Julián Álvarez,2023-24,10,6.191041,2,86
102965,GK,Sam Johnstone,2023-24,10,5.349623,1,90
103297,MID,Mohamed Salah,2023-24,10,9.701604,8,90
102851,MID,Son Heung-min,2023-24,10,7.648716,10,90
102834,MID,Douglas Luiz Soares de Paulo,2023-24,10,6.967505,7,89



=== Bench (4) ===


,pos,name,season_x,GW,expected_points,total_points,minutes
102855,DEF,Pedro Porro,2023-24,10,5.553708,2,90
102893,FWD,Yoane Wissa,2023-24,10,4.714238,2,75
103033,GK,Guglielmo Vicario,2023-24,10,5.180631,3,90
103238,MID,Hwang Hee-chan,2023-24,10,5.951396,7,90



Budget spent: 100.0 / 100.0M
Expected points (starters): 80.29
Expected points (bench * 0.1): 2.14 (raw bench 21.40)


In [256]:
pool['value'].values

array([ 4.5,  4.2,  4.5,  4.8,  5. ,  4.4,  5.5,  4. ,  4. ,  4.9,  4.4,
        4.3,  4.9,  4.5,  4.5,  4. ,  4. ,  4.9,  4.7,  5.4,  7.4,  5.4,
        4.5,  4.4,  3.9,  4.9,  4.5,  7.5,  5. ,  5.9,  4.4,  4. ,  3.9,
        5.5,  4.9,  6. ,  4.2,  4. ,  4.4,  4.8,  3.9,  4. ,  4.8,  4.4,
        4.5,  5.5,  4.5,  6.5,  4.4,  5. ,  4.8,  4.4,  4. ,  4.3,  4.4,
        4.7,  6.5,  5. ,  5.7,  4.4,  6.5,  4.5,  4.5,  6.8,  4.5,  4.5,
        4.3,  4.9,  5.4,  4. ,  4.4,  3.9,  5.1,  4.9,  4. ,  4.4,  4. ,
        5.4,  4.3,  4.4,  4.3,  4. ,  4.4,  4.8,  4.8,  4.5,  4. ,  5.6,
        4.4,  6.3,  4.7,  5. ,  4.9,  4.5,  5.5,  4. ,  5. ,  4.4,  4.5,
        4.5,  3.9,  4. ,  5.8,  8.3,  4.4,  5.8,  4.9,  6.7,  8.6,  5.2,
        3.9,  4.9,  4. ,  7.8,  3.9,  4.4,  4.5,  3.9,  4.5,  5.4,  4.5,
        4.8,  5.5,  4.3,  4.4,  3.9,  3.9,  5. ,  4.8,  4.9,  4. ,  4.3,
        3.9,  4.9,  5.9,  5.5,  5.5,  7.3,  4.5,  7.2,  5. ,  4.4,  4.4,
        5.4,  4.5,  4.9,  4.9,  3.9,  4.5,  4.7,  3